# Notebook to ingest data from ONS

---

In [1]:
# CMP259 - Ingestão de dados da ONS (SE/CO)

import requests
import pandas as pd
from datetime import date, timedelta
from pathlib import Path

pd.set_option("display.max_columns", 100)

BASE_DIR = Path("data_ons")
BASE_DIR.mkdir(exist_ok=True)


## Funções Auxiliares
---

In [ ]:
# Funções auxiliares

CKAN_BASE_URL = "https://dados.ons.org.br/api/3/action"
CARGA_API_BASE = "https://apicarga.ons.org.br/prd"
COD_AREACARGA_SECO = "SECO"  # Sudeste/Centro-Oeste, conforme dicionário de dados


def get_ckan_resources(dataset_id: str):
    """
    Busca metadados de um conjunto de dados no CKAN da ONS e retorna a lista de recursos.
    """
    resp = requests.get(f"{CKAN_BASE_URL}/package_show", params={"id": dataset_id})
    resp.raise_for_status()
    payload = resp.json()
    if not payload.get("success", False):
        raise RuntimeError(f"Erro CKAN para dataset {dataset_id}: {payload}")
    return payload["result"]["resources"]


def pick_parquet_resource_by_year(resources, year: int):
    """
    Escolhe recurso PARQUET cujo nome ou URL contenha o ano desejado.
    """
    year_str = str(year)
    candidates = [
        r
        for r in resources
        if r.get("format", "").upper() == "PARQUET"
        and (year_str in r.get("name", "") or year_str in r.get("url", ""))
    ]
    if not candidates:
        raise ValueError(f"Nenhum recurso PARQUET encontrado para ano {year}")
    # se houver mais de um, pega o primeiro
    return candidates[0]["url"]


def fetch_carga_api(endpoint: str,
                    dat_inicio: str,
                    dat_fim: str,
                    cod_areacarga: str = COD_AREACARGA_SECO) -> pd.DataFrame:
    """
    Chamada genérica para as APIs de carga (verificada ou programada).
    endpoint: 'cargaverificada' ou 'cargaprogramada'
    datas em formato YYYY-MM-DD
    """
    url = f"{CARGA_API_BASE}/{endpoint}"
    params = {
        "dat_inicio": dat_inicio,
        "dat_fim": dat_fim,
        "cod_areacarga": cod_areacarga,
    }
    r = requests.get(url, params=params)
    r.raise_for_status()
    data = r.json()

    # Estrutura flexível, pois o swagger pode mudar o wrapper
    if isinstance(data, dict):
        if "data" in data:
            rows = data["data"]
        elif "items" in data:
            rows = data["items"]
        else:
            # assume que a resposta já é uma lista de registros
            rows = data.get("result", data)
    else:
        rows = data

    return pd.json_normalize(rows)


def month_range(start: date, end: date):
    """
    Gera pares (inicio_mes, fim_mes) cobrindo o intervalo de forma contínua.
    """
    current = date(start.year, start.month, 1)
    while current <= end:
        if current.month == 12:
            next_month = date(current.year + 1, 1, 1)
        else:
            next_month = date(current.year, current.month + 1, 1)
        yield max(current, start), min(next_month - timedelta(days=1), end)
        current = next_month
